### Setup Segment Anything Model

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2

import torch
import torchvision
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor


## Get Device
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

## Setup SAM model
sam_checkpoint = "./models/sam_vit_h_4b8939.pth"
model_type = "vit_h"

## Register sam model
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

## Create a predictor for segment
predictor = SamPredictor(sam)

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    
def read_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def show_figure_point(image, input_point, input_label):
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    show_points(input_point, input_label, plt.gca())
    plt.axis('on')
    plt.show()

In [ ]:
## Read Image
image = read_image('./images/Chandra gupta obv.png')
image.shape

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(image)
plt.axis("off")

In [ ]:
# set image
predictor.set_image(image)

In [ ]:
# Add points for segmentation
input_point = np.array([[280, 650],[570, 290],[550, 335],[510, 310],[448, 255],[270,250], [300, 95], [350, 100],[320, 300], [400, 360], [230,330], [250, 550], [350, 550]])
input_label = np.array([1,1,1,1,1, 1,1, 1, 1, 0, 1, 1, 1])

In [ ]:
show_figure_point(image, input_point, input_label)

In [ ]:
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True,
)

In [ ]:
for i, (mask, score) in enumerate(zip(masks, scores)):
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    show_mask(mask, plt.gca())
    plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')
    plt.show()